
# CC3039 — Modelación y Simulación  
**Laboratorio 6 — Parte 2: Análisis de Estabilidad (Modelo SIR con dinámica vital)**
## Michelle Mejía 22596 Silvia Illescas 22376

In [1]:

# === Imports ===
import sympy as sp
import numpy as np

# Mostrar matrices completas
sp.init_printing(use_unicode=True)

# === Variables simbólicas ===
S, I = sp.symbols('S I', real=True)
beta, gamma, mu, N = sp.symbols('beta gamma mu N', positive=True, real=True)

# === Ecuaciones del modelo SIR con dinámica vital ===
# dS/dt = mu*N - beta*S*I - mu*S
# dI/dt = beta*S*I - (gamma + mu)*I
dS_dt = mu*N - beta*S*I - mu*S
dI_dt = beta*S*I - (gamma + mu)*I

f = sp.Matrix([dS_dt, dI_dt])
vars_vec = sp.Matrix([S, I])

# === Jacobiano simbólico ===
J = f.jacobian(vars_vec)
J


⎡-I⋅β - μ     -S⋅β    ⎤
⎢                     ⎥
⎣  I⋅β     S⋅β - γ - μ⎦

In [2]:

print("Jacobiano simbólico J(S, I):")
sp.pprint(J)


Jacobiano simbólico J(S, I):
⎡-I⋅β - μ     -S⋅β    ⎤
⎢                     ⎥
⎣  I⋅β     S⋅β - γ - μ⎦



## Autovalores simbólicos en el ELE

El **ELE** es \((S^*, I^*) = (N, 0)\). Sustituimos \(S=N\) y \(I=0\) en el Jacobiano y calculamos autovalores simbólicos.


In [3]:

J_ELE = J.subs({S: N, I: 0})
print("Jacobiano en ELE (S=N, I=0):")
sp.pprint(J_ELE)

# Autovalores simbólicos en el ELE
eigs_ELE = J_ELE.eigenvals()
print("\nAutovalores simbólicos del ELE:")
for ev, mult in eigs_ELE.items():
    print(f"  λ = {sp.simplify(ev)}  (multiplicidad {mult})")

# Demostración simbólica de que el segundo autovalor equivale a (gamma + mu)*(R0 - 1),
# con R0 = beta*N/(gamma + mu)
R0 = sp.simplify(beta*N/(gamma + mu))
lambda2 = sp.simplify(beta*N - (gamma + mu))
print("\nR0 (simbólico) =", R0)
print("λ2 (simbólico)  =", lambda2)
print("λ2 == (gamma + mu)*(R0 - 1) ? ->", sp.simplify(lambda2 - (gamma + mu)*(R0 - 1)) == 0)


Jacobiano en ELE (S=N, I=0):
⎡-μ     -N⋅β    ⎤
⎢               ⎥
⎣0   N⋅β - γ - μ⎦

Autovalores simbólicos del ELE:
  λ = N*beta - gamma - mu  (multiplicidad 1)
  λ = -mu  (multiplicidad 1)

R0 (simbólico) = N*beta/(gamma + mu)
λ2 (simbólico)  = N*beta - gamma - mu
λ2 == (gamma + mu)*(R0 - 1) ? -> True



## Parámetros numéricos (mismos que en la Parte 1)

Se utilizan los mismos parámetros indicados en la guía (R₀ > 1):
- \(N = 1000\)  
- \(\beta = 0.5/N\)  (normalizada por \(N\))  
- \(\gamma = 0.1\)  
- \(\mu = 0.02\)  

Con estos valores, \(R_0 = \frac{\beta N}{\gamma + \mu} > 1\).


In [5]:
# === Parámetros numéricos (todo float) ===
N_val = 1000.0
beta_val = 0.5 / N_val   # normalizada por N
gamma_val = 0.1
mu_val = 0.02

# Chequeo R0
R0_val = (beta_val * N_val) / (gamma_val + mu_val)
print(f"R0 = {R0_val:.6f}")

# === Equilibrio endémico (fórmulas en números puros) ===
# S* = (gamma + mu) / beta
# I* = mu * (N - S*) / (beta * S*)
S_end = (gamma_val + mu_val) / beta_val
I_end = mu_val * (N_val - S_end) / (beta_val * S_end)

print(f"S* = {S_end:.6f}, I* = {I_end:.6f}")

#     Sustituimos parámetros y (S*, I*) y luego convertimos a numpy
import sympy as sp
import numpy as np

S, I = sp.symbols('S I', real=True)
beta, gamma, mu, N = sp.symbols('beta gamma mu N', positive=True, real=True)
dS_dt = mu*N - beta*S*I - mu*S
dI_dt = beta*S*I - (gamma + mu)*I
J = sp.Matrix([dS_dt, dI_dt]).jacobian(sp.Matrix([S, I]))

# Dict de parámetros numéricos (todos float)
params_num = {N: N_val, beta: beta_val, gamma: gamma_val, mu: mu_val}

# Jacobiano numérico en el endémico
J_num_end_sym = J.subs(params_num).subs({S: S_end, I: I_end}).evalf()
print("\nJacobiano numérico en el equilibrio endémico:")
sp.pprint(J_num_end_sym, wrap_line=False)

# A numpy y autovalores numéricos
J_np = np.array(J_num_end_sym.tolist(), dtype=float)
eigvals_end = np.linalg.eigvals(J_np)
print("\nAutovalores numéricos en el equilibrio endémico:")
for ev in eigvals_end:
    print(ev)


R0 = 4.166667
S* = 240.000000, I* = 126.666667

Jacobiano numérico en el equilibrio endémico:
⎡-0.0833333333333333          -0.12        ⎤
⎢                                          ⎥
⎣0.0633333333333333   -1.38777878078145e-17⎦

Autovalores numéricos en el equilibrio endémico:
(-0.04166666666666668+0.07657603338440096j)
(-0.04166666666666668-0.07657603338440096j)



## Jacobiano numérico en el Equilibrio Endémico y sus autovalores


In [6]:

# Jacobiano simbólico con parámetros numéricos + (S_end, I_end)
J_num_end = J.subs(params_num).subs({S: S_end, I: I_end})

print("Jacobiano numérico en el equilibrio endémico:")
sp.pprint(sp.N(J_num_end, 6))

# Convertir a numpy y calcular autovalores numéricos
J_np = np.array(J_num_end.tolist(), dtype=np.float64)
eigvals_end = np.linalg.eigvals(J_np)

print("\nAutovalores numéricos del Jacobiano en el equilibrio endémico:")
for ev in eigvals_end:
    print(ev)


Jacobiano numérico en el equilibrio endémico:
⎡-0.0833333     -0.12    ⎤
⎢                        ⎥
⎣0.0633333   -1.38778e-17⎦

Autovalores numéricos del Jacobiano en el equilibrio endémico:
(-0.04166666666666668+0.07657603338440096j)
(-0.04166666666666668-0.07657603338440096j)



## Parte 2 — Preguntas de Análisis 

¿Qué representa ∂(dI/dt)/∂S?
Es la sensibilidad de la tasa de cambio de infectados respecto al número de susceptibles. En este modelo es βI: si I>0, al aumentar 𝑆 aumenta linealmente la fuerza de infección; si I=0, la sensibilidad es nula. 

ELE inestable cuando 𝑅0>1 y política “esperar y ver”
Si aparece un caso (perturbación pequeña), la infección crece (autovalor positivo) en lugar de disiparse. “Esperar y ver” es arriesgado porque permite que el brote despegue. 

Estabilidad del equilibrio endémico
“Estable” significa que perturbaciones se atenúan y el sistema regresa al nivel endémico. Si surge una variante y suben los casos, el sistema vuelve al equilibrio (posiblemente con oscilaciones amortiguadas) si los parámetros permanecen. 


Equilibrio estable en un MBA (estocástico)
No se ve como un punto fijo, sino como una nube de estados alrededor del equilibrio: el sistema orbita con ruido alrededor del atractor en el plano (S,I). 


Segundo autovalor del ELE
En el ELE: 𝜆2 =βN−(γ+μ). Con 𝑅0=𝛽𝑁/ 𝛾+𝜇

𝜆2=(𝛾+𝜇)(𝑅0−1).

Así, si 𝑅0>1⇒𝜆2>0, el ELE es inestable, como predice la teoría. 


Autovalores en el equilibrio endémico
Con los parámetros dados, los autovalores numéricos (que imprime el código) tienen parte real negativa, por lo que el equilibrio endémico es estable (atractor). 


Consistencia con la Parte 1
Con 𝑅0>1, las trayectorias no se quedan en el ELE: terminan en el endémico. Esto es coherente con el análisis: ELE inestable y endémico estable. 

Efecto de disminuir μ (↑ esperanza de vida) en 𝜆2 del ELE

𝜆2=(𝛾+𝜇)(𝑅0−1)=𝛽𝑁−(𝛾+𝜇). Si baja μ (manteniendo β,γ,N), λ2 disminuye ⇒ el crecimiento inicial del brote es más lento (sigue siendo positivo si 𝑅0>1, pero menor).


Uso de Gen AI 

Último prompt utilizado:

"Explícame de forma clara la Parte 2 (Jacobiano, autovalores en el ELE y en el equilibrio endémico) y dame el análisis/interpretación de los resultados; incluye definiciones breves de conceptos clave (ELE, estabilidad, 𝑅0) y conéctalo con lo observado en las trayectorias."

Por qué funcionó: El prompt pidió explícitamente qué conceptos definir (ELE, estabilidad, 𝑅0), qué cálculos presentar (Jacobiano y autovalores en ambos equilibrios) y qué análisis redactar (interpretación y vínculo con la Parte 1). Esa precisión reduce ambigüedad, garantiza reproducibilidad y facilita que las salidas (matrices, autovalores, conclusiones) queden claras y verificables.